In [1]:
import gym
from stable_baselines import DQN
from stable_baselines.deepq.policies import CnnPolicy

C:\Users\Falcone\anaconda3\envs\gymoa\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Falcone\anaconda3\envs\gymoa\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Falcone\anaconda3\envs\gymoa\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Falcone\anaconda3\envs\gymoa\lib\site-packages\tensorflow\

In [2]:
#Descobrindo o escopo de ações que um agente pode ter no ambiente
env=gym.make('PongNoFrameskip-v4')
print(env.action_space)
print(env.action_space.n)

Discrete(6)
6


In [3]:
#Definições das ações
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [4]:
#Gerando ações aleatorias
for i in range(10):
    print(env.action_space.sample())

4
1
5
5
1
2
1
0
5
4


In [5]:
#Visualizando um estado
env=gym.make('PongNoFrameskip-v4')
obs=env.reset()
print(obs) #array de pixels

[[[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [109 118  43]
  [109 118  43]
  [109 118  43]]

 [[109 118  43]
  [109 118  43]
  [109 118  43]
  ...
  [109 118  43]
  [109 118  43]
  [109 118  43]]

 [[109 118  43]
  [109 118  43]
  [109 118  43]
  ...
  [109 118  43]
  [109 118  43]
  [109 118  43]]

 ...

 [[ 53  95  24]
  [ 53  95  24]
  [ 53  95  24]
  ...
  [ 53  95  24]
  [ 53  95  24]
  [ 53  95  24]]

 [[ 53  95  24]
  [ 53  95  24]
  [ 53  95  24]
  ...
  [ 53  95  24]
  [ 53  95  24]
  [ 53  95  24]]

 [[ 53  95  24]
  [ 53  95  24]
  [ 53  95  24]
  ...
  [ 53  95  24]
  [ 53  95  24]
  [ 53  95  24]]]


In [6]:
obs.shape

(210, 160, 3)

In [7]:
#Visualizando o agente interagindo com o ambiente
import time
env=gym.make('PongNoFrameskip-v4')
obs=env.reset()
for i in range(1000): #
    env.render()
    time.sleep(0.01)
    acao=env.action_space.sample() #acoes aleatorias
    obs, recompensa, terminou, info = env.step(acao)
    if terminou:
        print('Episodio finalizado depois de {} timesteps'.format(i+1))
        break
env.close()

C:\Users\Falcone\anaconda3\envs\gymoa\lib\site-packages\pyglet\image\codecs\wic.py:406: UserWarning: [WinError -2147417850] Não é possível alterar o modo de thread depois de o mesmo estar definido
  warnings.warn(str(err))


In [8]:
#Entendimento de alguns codigos antes de realizar o treinamento
from stable_baselines.common.cmd_util import make_atari_env
from stable_baselines.common.vec_env import VecFrameStack
env=make_atari_env('PongNoFrameskip-v4',num_env=3, seed=0) #ambiente pre-processado com dimensoes de pixels reduzida, etc. env=6 seriam 6 ambientes trabalhados em paralelo, porem deep Q-Learing nao suporta trabalho em paralelo.
env=VecFrameStack(env, n_stack=4) #4 frames agrupados para estabelecer um estado
obs=env.reset()
obs, recompensa, terminou, info = env.step([0,1,1]) #acao no formato de lista. Se fossem por exemplo 6 ambientes, seriam passados 6 valores nesta lista

In [9]:
obs.shape

(3, 84, 84, 4)

In [15]:
env=make_atari_env('PongNoFrameskip-v4',num_env=1, seed=0) #criando 1 ambiente
env=VecFrameStack(env, n_stack=4) #agrupando 4 frames juntos para determinar 1 estado

modelo = DQN(policy=CnnPolicy, env=env, gamma=0.99, learning_rate=0.0005, buffer_size=30000, 
            exploration_fraction=0.2, exploration_final_eps=0.05, exploration_initial_eps=1.0, 
            train_freq=4,batch_size=32, double_q=True, target_network_update_freq=1000,
             learning_starts=10000, verbose=1)
#train_freq=4: quantidade de steps para fazer 1 update da rede neural. 1 uptade após 4 iterações 
#double_q=True: duas redes neurais
#target_network_update_freq=1000: Frequencia de atualização da rede target. 1 atualizaçao a cada 1000 steps
#learning_starts=10000: quantidades de iteraçoes no aquecimento do treinamento

from stable_baselines.common.callbacks import CheckpointCallback
checkpoint_callback = CheckpointCallback(save_freq=1000, save_path='C:/Users/Falcone/Documents/0_MachineLearning/2022.09.21_AprendizadoPorReforco/2-JogoPong-DeepQ-Learning', name_prefix='DQN')

inicio=time.time()
modelo.learn(total_timesteps=20000, log_interval=10)
fim=time.time()
print('Tempo em minutos:',(fim-inicio)/60)

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 10       |
| mean 100 episode reward | -21      |
| steps                   | 6909     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 20       |
| mean 100 episode reward | -20.8    |
| steps                   | 15300    |
--------------------------------------
Tempo em minutos: 3.5790345986684162


In [11]:
#Visualizando o agente interagindo com o ambiente
import time
obs=env.reset()
terminou=False
recompensa_total=0

while terminou !=True:
    acao, estado = modelo.predict(obs)
    obs, recompensa, terminou, info = env.step(acao)
    recompensa_total+=recompensa
    env.render()
    time.sleep(0.1)
print('Recompensa total:', recompensa_total)
env.close()

Recompensa total: [-12.]
